# WebService Recommendator

We want that our recommendations can be tested on an interactive enviroment, so we need to make some updates on the recommendations function and mix it with html code.

In [5]:
import pandas as pd
import os, sys
import numpy as np
import seaborn as sns
import gc
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from gensim.models import Word2Vec

In [6]:
path = os.path.join('../Data/')
path_models = os.path.join('../Models/')

data = pd.read_csv(path + 'FinalItems/data_filtered.csv', sep = ';')
data_processed = pd.read_csv(path + 'FinalItems/data_final.csv')
data_processed['text'] = data_processed['text'].astype(str)

In [7]:
items_unique = data_processed.item_id.unique()
items_map = {i:val for i,val in enumerate(items_unique)}
items_map_inv = {val:i for i,val in enumerate(items_unique)}
items_map_text = data_processed.set_index('item_id_int').text.to_dict()

In [8]:
idx_customers_map = {i:val for i,val in enumerate(data_processed['customer_id_int'])}
idx_customers_df = pd.DataFrame({'idx': data_processed.index.values, 'customer_id_int': data_processed.customer_id_int})

In [9]:
data.head(3)

,item_id,availability_date,brand,category,name,price
0,A28233506,NaN,Woman Limited El Corte Inglés,"['Moda', 'Mujer', 'Abrigos']",Abrigo masculino con textura de mujer,"{'final': 199, 'currency': 'EUR'}"
1,A29054782,NaN,Woman Limited El Corte Inglés,"['Moda', 'Mujer', 'Abrigos']",Abrigo doble faz de mujer con cinturón a tono,"{'final': 149, 'currency': 'EUR'}"
2,A27354432,NaN,Woman El Corte Inglés,"['Moda', 'Mujer', 'Abrigos']",Abrigo largo de antelina de mujer Woman El Cor...,"{'final': 89.99, 'currency': 'EUR'}"


In [10]:
data_processed.head(3)

,date,item_id,brand,PRICE,customer_id,text,item_age,customer_id_int,item_id_int,brand_id,score,score_original,power_price,power_score,power_item_age,sqrt_price,sqrt_score,sqrt_item_age
0,20190101,A26036172,tintoretto,0.003198,0,vestido mujer flor lazada,0.0,0,0,0,0.001485,4,0.000010,0.000002,0.0,0.056548,0.038538,0.0
1,20190101,A26870590,fórmula joven,0.002260,0,vestido laminado mujer formula joven escote pico,0.0,0,1,1,0.001485,4,0.000005,0.000002,0.0,0.047539,0.038538,0.0
2,20190103,MP_0659870_3014,gabrielle,0.008837,1,abrigo mujer negro avalorios,0.0,1,2,2,0.000000,1,0.000078,0.000000,0.0,0.094005,0.000000,0.0


In [11]:
MAX_NB_WORDS = 30_000 #decided by cumsum wordcount plot (Script 01)
MAX_SEQUENCE_LENGTH = 24 #decided by max words in a product (Script 00)
EMBEDDING_DIM = 100 #Same dim as our W2V embedding

all_text = data_processed['text']
all_text = all_text.drop_duplicates (keep = False)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, )
tokenizer.fit_on_texts(all_text)

data_sequences = tokenizer.texts_to_sequences(data_processed['text'])
data_vec = pad_sequences(data_sequences, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8487 unique tokens.


In [12]:
model = load_model(path_models + 'candidate_generation')
global graph
graph = tf.get_default_graph() 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


## Predictions

In [13]:
def recommend(customer, N = 5):
    try:
        _data = data_processed[data_processed['customer_id_int'] == customer]
        _data_vec = data_vec[_data.index]
        with graph.as_default():
            _pred = model.predict([_data['customer_id_int'], _data['item_id_int'], 
                                 _data['brand_id'], _data['PRICE'],
                                 _data_vec, _data['item_age'], _data['score'],
                                 _data['power_price'], _data['power_score'], _data['power_item_age'],
                                 _data['sqrt_price'], _data['sqrt_score'], _data['sqrt_item_age']],
                                 verbose = 1)
        _pred = pd.DataFrame(_pred)
        _pred['customer_id_int'] = customer
        _pred = _pred.groupby(['customer_id_int']).max()
        del _pred.index.name
        
    #########################################################

        print('\n' + '=='*30 + '\n')
        print(f'==> Top {N} Recommended items to Customer {customer}: ')
        print(f'\nThe customer {customer} has bought this items: ')
        print('\n' + '=='*30 + '\n')
        interacted_items = data_processed[['text', 'score_original']][data_processed['customer_id_int'] == customer].groupby('text')\
                            .sum().reset_index().sort_values(['score_original'], ascending = False)
        print('\n'.join([str(i+1) + str(' - ') + str(x) for i, x in enumerate(interacted_items['text'].values[0:20])]))
        top = _pred.values.reshape(_pred.shape[1]).argsort()[-N:][::-1] #items positions
        print('\n====================== IDs DE PRODUCTOS RECOMENDADOS ==============')
        print([items_map[item] for item in top])
        print ("\n===================== PRODUCTOS RECOMENDADOS =====================")
        print('\n'.join([str(i+1) + str(' - ') + str(items_map_text[x]) for i, x in enumerate(top)]))
        print ("==================================================================")
    except:
        print(f'\nThe customer {customer} does not exist')

In [14]:
CUSTOMER = 128 #128 #270
recommend(customer = CUSTOMER, N = 10)

11/11 [==============================] - 0s 9ms/sample


==> Top 10 Recommended items to Customer 128: 

The customer 128 has bought this items: 


1 - chaqueta punto hombre cuello alto
2 - harry_potter caliz fuego dvd
3 - harry_potter camara secreta dvd
4 - harry_potter coleccion hogwarts dvd bluray
5 - harry_potter orden fenix dvd
6 - harry_potter piedra filosofal dvd
7 - harry_potter prisionero azkaban dvd
8 - harry_potter reliquia muerte parte 3d bluray copia digital
9 - jersey hombre cuello redondo
10 - jersey hombre gris oscuro cuello caja
11 - pack harry_potter coleccion completa bluray

====================== IDs DE PRODUCTOS RECOMENDADOS ==============
['A24965927', 'A16759315', 'A28030903', 'A28486573', 'A4690016', 'A26912482', 'MP_0538405_N0YGPB197', 'A27251564', 'A29234841', 'A28744471']

===================== PRODUCTOS RECOMENDADOS =====================
1 - jersey hombre cuello redondo
2 - chaqueta punto hombre cuello alto
3 - pack harry_potter coleccion completa bluray
4 

### Html Recommendations

We mix our recommendtions output with html code to feed our web service

In [15]:
def header(text, color='black', gen_text=None, size = 54):
    """Create an HTML header"""

    if gen_text:
        raw_html = f'<h1 style="margin-top:16px;color: {color};font-size:{size}px"><center>' + str(
            text) + '<span style="color: #00BFFF">' + str(gen_text) + '</center></h1>'
    else:
        raw_html = f'<h1 style="margin-top:12px;color: {color};font-size:{size}px"><center>' + str(
            text) + '</center></h1>'
    return raw_html

def box(text):
    """Create an HTML box of text"""
    raw_html = '<div style="border-bottom:1px inset black;border-top:1px inset black;padding:8px;font-size: 21px;">' + str(
            text) + '</div>'
    return raw_html

In [23]:
def recommend_webservice(customer  , N = 5):
    items_bought = ''
    items_rec = ''
    text = ''
    html = ''
    
    try:
        _data = data_processed[data_processed['customer_id_int'] == customer]
        _data_vec = data_vec[_data.index]
        _pred = model.predict([_data['customer_id_int'], _data['item_id_int'], 
                             _data['brand_id'], _data['PRICE'],
                             _data_vec, _data['item_age'], _data['score'],
                             _data['power_price'], _data['power_score'], _data['power_item_age'],
                             _data['sqrt_price'], _data['sqrt_score'], _data['sqrt_item_age']],
                             batch_size = 1, verbose = 0)
        _pred = pd.DataFrame(_pred)
        _pred['customer_id_int'] = customer
        _pred = _pred.groupby(['customer_id_int']).max()
        del _pred.index.name
        
    #########################################################

        interacted_items = data_processed[['text', 'score_original']][data_processed['customer_id_int'] == customer].groupby('text')                            .sum().reset_index().sort_values(['score_original'], ascending = False)
        items_bought += '<br/>'.join([str(i+1) + str(' - ') + str(x) for i, x in enumerate(interacted_items['text'].values[0:20])])
        top = _pred.values.reshape(_pred.shape[1]).argsort()[-N:][::-1] #items positions
        items_rec += '<br/>'.join([str(i+1) + str(' - ') + str(items_map_text[x]) for i, x in enumerate(top)])

    except Exception as e:
        print(f'Exception: {e}')
        print(f'\nThe customer {customer} does not exist')

    html = header('', color = 'black', gen_text = f'Customer {customer} Recommendations')  
    html += header(text = f'Items bought by Customer {customer}:', size = 40)
    html += box(items_bought)
    html += header(text = f'Top {N} Items recommended to Customer {customer}:', size = 40)
    html += box(items_rec)
        
    return f'<div>{html}</div>'



In [24]:
CUSTOMER = 128 #128 #270
recommend_webservice(customer = CUSTOMER, N = 10)

'<div><h1 style="margin-top:16px;color: black;font-size:54px"><center><span style="color: #00BFFF">Customer 128 Recommendations</center></h1><h1 style="margin-top:12px;color: black;font-size:40px"><center>Items bought by Customer 128:</center></h1><div style="border-bottom:1px inset black;border-top:1px inset black;padding:8px;font-size: 21px;">1 - chaqueta punto hombre cuello alto<br/>2 - harry_potter caliz fuego dvd<br/>3 - harry_potter camara secreta dvd<br/>4 - harry_potter coleccion hogwarts dvd bluray<br/>5 - harry_potter orden fenix dvd<br/>6 - harry_potter piedra filosofal dvd<br/>7 - harry_potter prisionero azkaban dvd<br/>8 - harry_potter reliquia muerte parte 3d bluray copia digital<br/>9 - jersey hombre cuello redondo<br/>10 - jersey hombre gris oscuro cuello caja<br/>11 - pack harry_potter coleccion completa bluray</div><h1 style="margin-top:12px;color: black;font-size:40px"><center>Top 10 Items recommended to Customer 128:</center></h1><div style="border-bottom:1px inset 

---

#### WebService

We are ready to make recommendations with our WebService, go to [WebService.py](http://localhost:8888/edit/Google%20Drive/PROYECTOS/TFM_Motor%20de%20Recomendacion/Master/99_FinalModel/Scripts/WebService.py) script